# 爬取百城的经纬度坐标
以该市人民政府所在地代表该市

In [ ]:
# 载入必要的模块
import json
import requests
import numpy as np
from pandas import Series, DataFrame
import pandas as pd

# 创建一个空的DataFrame，并选择保留'name', 'address', 'location'三列信息
df = {'name': [], 'address': [], 'location': []}
result = DataFrame(df)

col = ['name', 'address', 'location']

In [ ]:
# 读取保存有百城名称的csv文件，并将DataFrame转化为Series
city = pd.read_csv("../data/cities.csv", encoding = 'gb18030')
city = Series(city['city'])

In [ ]:
# 将城市名转化为utf8编码，并在其后添加“市政府”的utf8编码
for i in range(len(city)):
    city[i] = repr(city[i].encode("UTF-8")).replace('\\x', '%').strip("'") + '%e5%b8%82%e6%94%bf%e5%ba%9c'

In [ ]:
# for-loop，爬取百城坐标
for i in range(len(city)):
    url = 'http://api.map.baidu.com/place/v2/search?query=' + city[i] + '&page_size=5&page_num=0&scope=1&region=全国&output=json&ak=7QP1yCu8hlfwV3QV1mAhkmpmp0kR2K8d'
    resp = requests.get(url)
    data = json.loads(resp.text)
    txt = DataFrame(data['results']).ix[:0, ['name', 'location', 'address']]
    result = pd.concat([result, txt], axis = 0, ignore_index=True)

In [ ]:
# 预览爬取的百城坐标
result

In [ ]:
# 将result写入名为raw_coordinate的csv文件中
result.to_csv("../data/raw_coordinate.csv",encoding='gb18030')

In [ ]:
# 处理'location'列，将经度和纬度分离，并写入名为final_coordinate的csv文件中

# 选取location列
location = result['location']

# 创建空列表，后续分离的经纬度将会存放在该列表中
location_result = []

# 将经度和纬度分离，并进行必要的字符串处理
for raw_location in location:
    # 创建一个临时的列表容器
    temp_result = []
        
    # 将经纬度分离
    temp_coordinate = str(raw_location).split(",")
    
    # 进行必要的字符串处理，将结果放入temp_result中
    for loc in temp_coordinate:
        loc = loc.strip("{u'lat': ").strip(" u'lng': ").strip("}")
        temp_result.append(loc)
    
    location_result.append(temp_result)

# 将存放分离的经纬度的列表转换为DataFrame
final_coordinate = DataFrame(location_result)

# 将存放分离的经纬度的DataFrame，写入名为final_coordinate的csv文件中
final_coordinate.to_csv("../data/final_coordinate.csv",encoding='gb18030')

In [ ]:
# 对个别存在问题的城市坐标，进行手工爬取
url = 'http://api.map.baidu.com/place/v2/search?query=无锡市人民政府&page_size=5&page_num=0&scope=1&region=全国&output=json&ak=7QP1yCu8hlfwV3QV1mAhkmpmp0kR2K8d'
resp = requests.get(url)
data = json.loads(resp.text)
txt = DataFrame(data['results'], columns = col)

In [ ]:
# 预览手工爬取的城市坐标
txt